In [1]:
# Loading libraries
lib_load <- function(){
  
  load.lib = c('tidyverse', "tidytext", "tm", "SnowballC", 'qdap', 'textclean', 'stringr', 'tokenizers', 'RColorBrewer', 'wordcloud', 
               'wordcloud2', 'RWeka')
  #We create a variable where we indicating that we select the ones that we do not have installed.                     
  install.lib <- load.lib[!load.lib %in% installed.packages()]
  #With a loop we install the libraries that we do not have.
  for(lib in install.lib) install.packages(lib,dependences=TRUE)
  #Load the libraries
  sapply(load.lib,require,character=TRUE)
}
lib_load()

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
also installing the dependencies ‘hunspell’, ‘tokenizers’, ‘janeaustenr’, ‘stopwords’

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
also installing the dependencies ‘dtt’, ‘syuzhet’, ‘english’, ‘lexicon’, ‘mgsub’, ‘textshape’

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.1
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag

tidyverse     tidytext           tm    SnowballC         qdap    textclean 
        TRUE         TRUE         TRUE         TRUE         TRUE         TRUE 
     stringr   tokenizers RColorBrewer    wordcloud   wordcloud2        RWeka 
        TRUE         TRUE         TRUE         TRUE         TRUE         TRUE

In [2]:
metoo_tweets = read.csv('MeTooIndia - MeTooIndia.csv')

In [3]:
glimpse(metoo_tweets)

Observations: 604
Variables: 22
$ X                <int> 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15…
$ screen_name      <fct> IndiaMeToo, IndiaMeToo, SayftyCom, IndiaMeToo, Rajiv…
$ username         <fct> #MeTooIndia, #MeTooIndia, Sayfty.com, #MeTooIndia, R…
$ user_id          <dbl> 1.049247e+18, 1.049247e+18, 1.266039e+09, 1.049247e+…
$ tweet_id         <dbl> 1.066921e+18, 1.263417e+18, 1.257365e+18, 1.260936e+…
$ tweet_url        <fct> /IndiaMeToo/status/1066921296129417216, /IndiaMeToo/…
$ timestamp        <fct> 2018-11-26 5:07:14, 2020-05-21 10:30:49, 2020-05-04 …
$ timestamp_epochs <int> 1543208834, 1590057049, 1588614120, 1589465456, 1589…
$ text             <fct> "On days following the International Day to Eliminat…
$ text_html        <fct> "<p class=\"TweetTextSize TweetTextSize--normal js-t…
$ links            <fct> [], ['https://www.thequint.com/news/india/rape-ram-t…
$ hashtags         <fct> "['Metoo']", "[]", "['consenteducation']", "['domest…
$ has_media        <

In [4]:
tweets = metoo_tweets%>%
    select(text)%>%
    distinct() # checking and removing duplicate tweets

In [5]:
dim(tweets) # we have 601 unique tweets

[1] 601   1

# Text cleaning

In [6]:
text = tweets[,1]
clean_text = function(x)
{
x = gsub("rt", "", x) # remove Retweet
x = gsub("@\\w+", "", x) # remove at(@)
x = gsub("#\\w+", "", x)# remove # tags
x = gsub("http\\w+", "", x)  # remove links http
x = gsub("[[:punct:]]", "", x) # remove punctuation
x = gsub("[[:digit:]]", "", x) # remove numbers/Digits
x = gsub("[ |\t]{2,}", "", x) # remove tabs
x = gsub("^ ", "", x)  # remove blank spaces at the beginning
x = gsub(" $", "", x) # remove blank spaces at the end
return(x)
}

text = clean_text(text)

# I want to correct some mistakes in the text
text = gsub('ConsentEducation', 'Consent Education', text)
text = gsub('\n', '', text)
text = str_replace_all(text, 'pic?twittercom\\w+', '')
text = str_replace_all(text, 'twittercom\\w+', '')

In [7]:
# tm cleaning function 
tm_clean <- function(text) {
  corpus = VCorpus(VectorSource(text))
  corpus = tm_map(corpus, content_transformer(tolower))
  corpus = tm_map(corpus, removeNumbers)
  corpus = tm_map(corpus, removePunctuation)
  corpus = tm_map(corpus, removeWords, stopwords('en'))
  corpus = tm_map(corpus, stemDocument)
  corpus = tm_map(corpus, stripWhitespace)
  return(corpus) 
}

text_corpus = tm_clean(text)

In [8]:
# creating a TermDocumentMatrix
dv_tdm = TermDocumentMatrix(text_corpus)
dv_matrix = as.matrix(dv_tdm) # converting to matrix

In [9]:
#Gennereating term frequence and plotting 
term_frequency = rowSums(dv_matrix)
term_frequency = sort(term_frequency, decreasing = TRUE)
df <- data.frame(word = names(term_frequency),freq=term_frequency)


barplot(term_frequency[1:25],           
        col = "blue", las = 2)

In [10]:
wordcloud2(data=df, size=1, color='random-dark')

HTML widgets cannot be represented in plain text (need html)

In [11]:
# lets see the bigrams

 # Define bigram tokenizer 
tokenizer <- function(x)     
    NGramTokenizer(x, Weka_control(min = 2, max = 2)) 

In [12]:
# Make a bigram TDM 
bigram_tdm <- TermDocumentMatrix(     
    text_corpus,      
    control = list(tokenize = tokenizer))

In [13]:
bigram_matrix = as.matrix(bigram_tdm)
bigram_term_frequency = rowSums(bigram_matrix)
bigram_term_frequency = sort(bigram_term_frequency, decreasing = TRUE)


In [24]:
bigrams_df = data.frame(words = names(bigram_term_frequency), freq = bigram_term_frequency)
head(bigrams_df)

,words,freq
,<fct>,<dbl>
sexual harass,sexual harass,36
domest violenc,domest violenc,12
sexual assault,sexual assault,11
sexual violenc,sexual violenc,8
transgend peopl,transgend peopl,8
mj akbar,mj akbar,7


In [28]:
bigrams_df%>%
    top_n(20)%>%
ggplot(aes(x = reorder(words, freq), y = freq))+
    geom_col(fill = "royalblue")+
    scale_y_continuous()+
    coord_flip()+
    labs(title = "Bigrams plot for MeToo data",
        x = "Words",
        y = "Term frequency")

Selecting by freq
